<a href="https://colab.research.google.com/github/mythgotham007/hello-world/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 235kB 18.4MB/s 
     |████████████████████████████████| 2.2MB 16.2MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.0-cp37-none-any.whl size=365386 sha256=9d2bd4936a12820fa06fe669b1ff8a8442fc4c097415e5b368ef90658a52bbac
  Stored in directory: /root/.cache/pip/wheels/ce/fb/df/37cd43d1433e65b2d3417a71438404d0eeb4fbbfa40730931c
Successfully built torch-geometric


In [ ]:
!pip install torch_sparse

  Created wheel for torch-sparse: filename=torch_sparse-0.6.9-cp37-cp37m-linux_x86_64.whl size=432305 sha256=b748827211961e48cc3efee2e8bac66a64d16dc936e10be5311b8cb4d3c87dcc
  Stored in directory: /root/.cache/pip/wheels/6c/9f/3e/8813c1f7e87d12b01b830d3506e7f3f6b044d9d53769057ddb
Successfully built torch-sparse


In [ ]:
!pip install torch_scatter

  Created wheel for torch-scatter: filename=torch_scatter-2.0.6-cp37-cp37m-linux_x86_64.whl size=292562 sha256=25082cbe0d52ef70b96835bbc3809a3eda833b4987273a742697149e46406d8b
  Stored in directory: /root/.cache/pip/wheels/d6/bf/04/7c9e0c1466d37548e5c7c7c2daee2b7f90ce616f102c68bf07
Successfully built torch-scatter


In [ ]:
!unzip ./exp_ER_GNN_Steiner_TSP_50.zip

Archive:  ./exp_ER_GNN_Steiner_TSP_50.zip
   creating: exp_ER_GNN_Steiner_TSP_50/
  inflating: exp_ER_GNN_Steiner_TSP_50/graph_ER_50_1_L_50_184.txt  
   creating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/
 extracting: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/error_57.dat  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/graph_ER_50_1_L_50_3_output.txt  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/graph_ER_50_1_L_50_92_output.txt  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/graph_ER_50_1_L_50_150_output.txt  
 extracting: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/error_53.dat  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/output_156.dat  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/output_13.dat  
 extracting: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/error_62.dat  
  inflating: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/output_187.dat  
 extracting: exp_ER_GNN_Steiner_TSP_50/log_folder_exact/error_183.dat  
 extracting: exp_

In [ ]:
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

folder_name = "./exp_ER_GNN_Steiner_TSP_50/"
file_size = 150
file_names = ["graph_ER_50_1_L_50_"+str(i)+".txt" for i in range(file_size)]
sol_file_names = ["log_folder_exact/graph_ER_50_1_L_50_"+str(i)+"_output.txt" for i in range(file_size)]

def is_comment(x):
 if x[0]=='#':
  return True
 return False

def take_input(input_file):
 file = open(input_file,"r")
 #print("File name: "+input_file)
 while True:
  l = file.readline()
  #print(l)
  if not is_comment(l):
   break
 m = int(l)
 edge_list = list()
 for i in range(m):
    while True:
     l = file.readline()
     if len(l) == 0:
      break
     if not is_comment(l):
      break
    t_arr1 = []
    t_arr2 = l.split()
    if(len(t_arr2)<3):break
    t_arr1.append(int(t_arr2[0])-1)
    t_arr1.append(int(t_arr2[1])-1)
    #t_arr1.append(int(t_arr2[0]))
    #t_arr1.append(int(t_arr2[1]))
    #t_arr1.append(int(t_arr2[2]))
    t_arr1.append(float(t_arr2[2]))
    edge_list.append(t_arr1)

 levels = int(file.readline())
 tree_ver=[]
 #tree_ver = [(int(x)-1) for x in raw_input().split()]
 for l in range(levels):
  #print "Steiner tree vertices of level "+str(l+1)+":"
  tree_ver.append([(int(x)-1) for x in file.readline().split()])
  #tree_ver.append([(int(x)) for x in file.readline().split()])

 file.close()
 return edge_list, tree_ver

def prepare_gcn_data(folder_name, file_names, sol_file_names):
  node_id_additive = 0
  total_edges = []
  total_nodes = []
  total_labels = []
  data_list = []

  for i, (file_name, sol_file_name) in enumerate(zip(file_names, sol_file_names)):
    G = nx.Graph()
    edge_list, tree_ver = take_input(folder_name + file_name)
    for e in edge_list:
      G.add_edge(e[0], e[1])
    max_id = max([max(u, v) for u, v, _ in edge_list])
    min_id = min([min(u, v) for u, v, _ in edge_list])
    #edge_list = [[node_id_additive+u, node_id_additive+v, i] for u, v, _ in edge_list]
    '''
    edge_list1 = [[node_id_additive+u, node_id_additive+v, G.degree[u], G.degree[v], i] for u, v, _ in edge_list]
    edge_list2 = [[node_id_additive+v, node_id_additive+u, G.degree[v], G.degree[u], i] for u, v, _ in edge_list]
    '''
    edge_list1 = [[u, v, G.degree[u], G.degree[v], i] for u, v, _ in edge_list]
    edge_list2 = [[v, u, G.degree[v], G.degree[u], i] for u, v, _ in edge_list]
    #node_list = [[node_id_additive+u, 1] if u in tree_ver[0] else [node_id_additive+u, 0] for u in range(max_id+1)]
    #node_list = [[node_id_additive+u, 1, i, G.degree[u]] if u in tree_ver[0] else [node_id_additive+u, 0, i, G.degree[u]] for u in range(max_id+1)]
    node_list = [[u, 1, i, G.degree[u]] if u in tree_ver[0] else [u, 0, i, G.degree[u]] for u in range(max_id+1)]
    sol_edge_list, sol_tree_ver = take_input(folder_name + sol_file_name)
    #labels = [[0] for u in range(max_id+1)]
    #labels = [[0, 1] for u in range(max_id+1)]
    labels = [0 for u in range(max_id+1)]
    for u, v, w in sol_edge_list:
      '''
      labels[u] = [1]
      labels[v] = [1]
      '''
      #labels[u] = [1, 0]
      #labels[v] = [1, 0]
      labels[u] = 1
      labels[v] = 1
    node_id_additive += (max_id+1)
    #total_edges += edge_list
    total_edges += edge_list1
    total_edges += edge_list2
    total_nodes += node_list
    total_labels += labels

    x = torch.tensor(node_list, dtype=torch.float)
    y = torch.tensor(labels, dtype=torch.long)
    edge_index_forward = [arr[0] for arr in edge_list1]
    edge_index_backward = [arr[0] for arr in edge_list2]
    edge_list_top = edge_index_forward + edge_index_backward
    edge_index_forward = [arr[1] for arr in edge_list1]
    edge_index_backward = [arr[1] for arr in edge_list2]
    edge_list_bot = edge_index_forward + edge_index_backward
    edge_index = torch.tensor([edge_list_top, edge_list_bot], dtype=torch.long)
    edge_weight = torch.tensor([1 for _ in range(len(edge_list_top))], dtype=torch.float)
    data = Data(x=x, y=y, edge_index=edge_index, edge_weight=edge_weight)
    data_list.append(data)
  #return total_edges, total_nodes, total_labels
  loader = DataLoader(data_list, batch_size=1)
  return loader

loader = prepare_gcn_data(folder_name, file_names, sol_file_names)


In [ ]:
'''
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
'''

#use the following format
'''
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)

edge_index = torch.tensor([[0, 2, 1, 0, 3],
                           [3, 1, 0, 1, 2]], dtype=torch.long)

edge_weight = torch.tensor([1, 1, 1, 1, 1], dtype=torch.float)


data = Data(x=x, y=y, edge_index=edge_index, edge_weight=edge_weight)
'''
# x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.int)
#y = torch.tensor([0, 1, 0, 1], dtype=torch.int)

#x2 = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
'''
x2 = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.int)
y2 = torch.tensor([0, 1, 0, 1], dtype=torch.float)

edge_index2 = torch.tensor([[0, 2, 1, 0, 3],
                           [3, 1, 0, 1, 2]], dtype=torch.long)


data2 = Data(x=x2, y=y2, edge_index=edge_index2)

data_list = [data, data2]
loader = DataLoader(data_list, batch_size=1)
'''

'\nx2 = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.int)\ny2 = torch.tensor([0, 1, 0, 1], dtype=torch.float)\n\nedge_index2 = torch.tensor([[0, 2, 1, 0, 3],\n                           [3, 1, 0, 1, 2]], dtype=torch.long)\n\n\ndata2 = Data(x=x2, y=y2, edge_index=edge_index2)\n\ndata_list = [data, data2]\nloader = DataLoader(data_list, batch_size=1)\n'

In [ ]:
'''
dataset = dataset.shuffle()
train_dataset = dataset[:1]
val_dataset = dataset[1:]
test_dataset = dataset[1:]
len(train_dataset), len(val_dataset), len(test_dataset)
'''
print(type(loader))
dataset = loader.dataset
#dataset = dataset.shuffle()
'''
train_dataset = dataset[:1]
val_dataset = dataset[1:]
test_dataset = dataset[1:]
len(train_dataset), len(val_dataset), len(test_dataset)
'''
train_dataset = dataset
print("Training size:", len(train_dataset))

<class 'torch_geometric.data.dataloader.DataLoader'>
Training size: 150


In [ ]:
'''
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(type(dataset))
print(dataset.num_node_features)
print(dataset.num_classes)
print(dataset.get(0).x.shape)
'''

"\nfrom torch_geometric.datasets import Planetoid\n\ndataset = Planetoid(root='/tmp/Cora', name='Cora')\nprint(type(dataset))\nprint(dataset.num_node_features)\nprint(dataset.num_classes)\nprint(dataset.get(0).x.shape)\n"

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.emb = AtomEncoder(4)
        #self.conv1 = GCNConv(dataset.num_node_features, 16)
        #self.conv2 = GCNConv(16, dataset.num_classes)
        '''
        self.conv1 = GCNConv(2, 16)
        self.conv2 = GCNConv(16, 2)
        '''
        self.conv1 = GCNConv(4, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        #x, edge_index = data.x, data.edge_index
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight

        #x = self.emb(x)

        #x = self.conv1(x, edge_index)
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        #x = self.conv2(x, edge_index)
        x = self.conv2(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

In [ ]:
#!pip install ogb

In [ ]:
for data in train_dataset:
  #print(type(data))
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = Net().to(device)
  #data = dataset[0].to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  print(data)
  model.train()
  for epoch in range(200):
      #print("epoch", epoch)
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out, data.y)
      loss.backward()
      optimizer.step()

Data(edge_index=[2, 402], edge_weight=[402], x=[50, 4], y=[50])
Data(edge_index=[2, 340], edge_weight=[340], x=[50, 4], y=[50])
Data(edge_index=[2, 346], edge_weight=[346], x=[50, 4], y=[50])
Data(edge_index=[2, 330], edge_weight=[330], x=[50, 4], y=[50])
Data(edge_index=[2, 380], edge_weight=[380], x=[50, 4], y=[50])
Data(edge_index=[2, 394], edge_weight=[394], x=[50, 4], y=[50])
Data(edge_index=[2, 398], edge_weight=[398], x=[50, 4], y=[50])
Data(edge_index=[2, 404], edge_weight=[404], x=[50, 4], y=[50])
Data(edge_index=[2, 426], edge_weight=[426], x=[50, 4], y=[50])
Data(edge_index=[2, 346], edge_weight=[346], x=[50, 4], y=[50])
Data(edge_index=[2, 356], edge_weight=[356], x=[50, 4], y=[50])
Data(edge_index=[2, 338], edge_weight=[338], x=[50, 4], y=[50])
Data(edge_index=[2, 390], edge_weight=[390], x=[50, 4], y=[50])
Data(edge_index=[2, 436], edge_weight=[436], x=[50, 4], y=[50])
Data(edge_index=[2, 374], edge_weight=[374], x=[50, 4], y=[50])
Data(edge_index=[2, 376], edge_weight=[3

In [ ]:
test_file_names = ["graph_ER_50_1_L_50_"+str(i)+".txt" for i in range(151, 200)]
test_sol_file_names = ["log_folder_exact/graph_ER_50_1_L_50_"+str(i)+"_output.txt" for i in range(151, 200)]

test_loader = prepare_gcn_data(folder_name, test_file_names, test_sol_file_names)

In [ ]:
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)
    #return labels
    return predictions

labels = evaluate(test_loader)

In [ ]:
import numpy as np
import networkx as nx

def generate_solution_graphs(prediction, folder_name, file_names, sol_file_names):
 test_graph_edges, test_solution_edges, test_terminals = [], [], []
 connected_graphs = 0
 disconnected_graphs = 0
 disconnected_opt_solutions = 0
 rat_arr = []
 node_id_additive = 0

 for j, (file_name, sol_file_name) in enumerate(zip(file_names, sol_file_names)):
  edge_list, tree_ver = take_input(folder_name + file_name)
  sol_edge_list, sol_tree_ver = take_input(folder_name + sol_file_name)
  max_id = max([max(u, v) for u, v, _ in edge_list])
  min_id = min([min(u, v) for u, v, _ in edge_list])
  #pred = pr[node_id_additive:node_id_additive+max_id+1]
  pred = prediction[j]
  pred = pred.argmax(axis=1)
  sorted_ind = np.argsort(pred)
  #print(pred)
  #print(sorted_ind)
  pred = np.round(pred)
  opt = len(sol_edge_list)
  G = nx.Graph()
  for e in edge_list:
   u, v, w = e
   G.add_edge(u, v)
  V = []
  '''
  for i, val in enumerate(pred):
   if val==1:
    V.append(i)
    #G.add_edge(*id_to_edge[i])
  '''
  for terminal in tree_ver[0]:
   if terminal not in V:
    V.append(terminal)
  S = G.subgraph(V)
  #'''
  #is_disconnected = False
  #if not nx.is_connected(S):
  # is_disconnected = True
  is_disconnected = not nx.is_connected(S)
  k = len(sorted_ind)-1
  #print("k", k)
  while not nx.is_connected(S):
   u = sorted_ind[k]
   #print(u)
   if u not in V:
    V.append(u)
    S = G.subgraph(V)
   k -= 1
  #if is_disconnected:
  # mst=nx.minimum_spanning_edges(S)
  # apprx = len(list(mst))
  # if opt==apprx:
  #  disconnected_opt_solutions += 1
  mst=nx.minimum_spanning_edges(S)
  apprx = len(list(mst))
  if opt==apprx:
    disconnected_opt_solutions += 1
  #'''
  if nx.is_connected(S):
   connected_graphs += 1
   mst=nx.minimum_spanning_edges(S)
   apprx = len(list(mst))
   rat_arr.append(apprx/opt)
  else:
   disconnected_graphs += 1

  node_id_additive += (max_id+1)

 print("No. of connected graphs:", connected_graphs)
 total_rat = 0
 for rat in rat_arr:
  total_rat += rat
 avg_rat = total_rat/connected_graphs
 print("average ratio:", avg_rat)
 print("No. of disconnected graphs:", disconnected_graphs)
 print("No. of disconnected graphs become optimal after connecting:", disconnected_opt_solutions)

In [ ]:
generate_solution_graphs(labels, folder_name, test_file_names, test_sol_file_names)

No. of connected graphs: 49
average ratio: 1.086530612244898
No. of disconnected graphs: 0
No. of disconnected graphs become optimal after connecting: 36
